In [1]:
from bs4 import BeautifulSoup
import urllib3
import numpy as np
import pickle

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
urls = ['https://sfbay.craigslist.org/search/apa?sort=date']
num = np.arange(120, 2881, 120)
for n in num:
    urls.append('https://sfbay.craigslist.org/search/apa?s='+str(n)+'&sort=date')
total_added = 0

In [3]:
def make_soup(url):
    http = urllib3.PoolManager()
    r = http.request("GET", url)
    return(BeautifulSoup(r.data, 'html.parser'))

In [14]:
urls

['https://sfbay.craigslist.org/search/apa?sort=date',
 'https://sfbay.craigslist.org/search/apa?s=120&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=240&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=360&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=480&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=600&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=720&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=840&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=960&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1080&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1200&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1320&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1440&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1560&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1680&sort=date',
 'https://sfbay.craigslist.org/search/apa?s=1800&sort=date',
 'https://sfbay.craigslist.org/search/a

In [15]:
listings = []
for url in urls:
    soup = make_soup(url)
    results = soup.findAll('li', class_ = 'result-row', recursive=True)
    for tag in results:
        cost = tag.find('span', class_="result-price")
        size = tag.find('span', class_="housing")
        place = tag.find('span', class_="result-hood")
        listing_dict = {'link': tag.a['href'], 
                        'id': tag['data-pid'], 
                        'title': tag.find('a', class_='result-title hdrlnk').get_text(),
                        'cost':  (cost.get_text() if cost else None),
                        'size': (size.get_text().strip() if size else None),
                        'place': (place.get_text().strip() if place else None),
                        'last_pos_time': tag.find('time', class_="result-date")['datetime']}
        listings.append(listing_dict)

In [16]:
with open('new_listings_2nd.pickle', 'wb') as file:
    pickle.dump(listings, file, protocol=pickle.HIGHEST_PROTOCOL)

Scraping each listing for more information

In [4]:
with open('listings.pickle', 'rb') as file:
    listings = pickle.load(file)

In [17]:
for i in range(len(listings)):
    soup_list = make_soup(listings[i]['link'])
    results = soup_list.find('section', class_ = "body")
    datetime = []
    for j in results.find_all('time', class_='date timeago'):
        datetime.append(j.get_text().strip())
    datetime = np.unique(datetime).tolist()
    lat = results.find('div', class_ = 'mapbox')
    lon = results.find('div', class_ = 'mapbox')
    text = results.find('section', class_ = "userbody")
    extra_info = results.find_all('p', class_='attrgroup')
    extra_info_text = ''
    for k in range(len(extra_info)):
        extra_info_text = extra_info_text + '\n\n' + extra_info[k].get_text().strip()
    
    listings[i]['latitude'] = (lat.div['data-latitude'] if lat else None)
    listings[i]['longitude'] = (lon.div['data-longitude'] if lon else None)
    listings[i]['text'] = (text.find('section').get_text().strip() if text else None)
    listings[i]['extra_info'] = extra_info_text
    listings[i]['datetime'] = datetime    

In [18]:
with open('new_listings_all_2nd.pickle', 'wb') as file:
    pickle.dump(listings, file, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
import pandas as pd

In [9]:
listings_df = pd.DataFrame(listings)

In [10]:
listings_df

,cost,datetime,extra_info,id,last_pos_time,latitude,link,longitude,place,size,text,title
0,$3300,[],,6724361608,2018-10-24 19:07,None,https://sfbay.craigslist.org/sby/apa/d/north-s...,None,(san jose north),2br -\n 1564ft2 -,None,North San Jose River Oaks Area 95134 Fully RE...
1,$2399,[],,6731996825,2018-10-24 19:06,None,https://sfbay.craigslist.org/nby/apa/d/get-hap...,None,(novato),2br -\n 1028ft2 -,None,"Get Happy, Lease Now At The Vineyard Apartment..."
2,$3300,"[2018-10-17 12:04am, 2018-10-26 9:42am]",\n\n2BR / 2.5Ba\n1564ft2\navailable oct 31\n\n...,6725407723,2018-10-24 19:06,37.402847,https://sfbay.craigslist.org/sby/apa/d/north-s...,-121.924755,(san jose north),2br -\n 1564ft2 -,QR Code Link to This Post\n\n\nCALL FOR IOEN H...,North San Jose in River Oaks Area 95134 Fully ...
3,$2875,[2018-10-24 7:02pm],\n\n2BR / 1Ba\n1750ft2\navailable dec 1\n\ntow...,6732006789,2018-10-24 19:06,37.901258,https://sfbay.craigslist.org/eby/apa/d/best-va...,-122.308644,(albany / el cerrito),2br -\n 1750ft2 -,QR Code Link to This Post\n\n\nFresh! Spacious...,Best Value! 2+ Bedroom Townhouse
4,$3695,[2018-10-24 7:02pm],\n\n1BR / 1Ba\n\napartment\n\nlaundry in bldg\...,6732006575,2018-10-24 19:06,37.796670,https://sfbay.craigslist.org/sfc/apa/d/large-1...,-122.438779,(marina / cow hollow),1br -,QR Code Link to This Post\n\n\nPlease contact:...,Large 1 bedroom in trendy Cow Hollow. Approxim...
5,$1649,"[2018-09-26 1:35pm, 2018-10-24 7:00pm]",\n\n1BR / 1Ba\n708ft2\n\napartment,6708276388,2018-10-24 19:05,38.008321,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,-121.864673,None,1br -\n 708ft2 -,QR Code Link to This Post\n\n\nDiamond Hillsid...,"❄ Pittsburg apartment for rent, Additional Sto..."
6,$2443,[],,6732006200,2018-10-24 19:05,None,https://sfbay.craigslist.org/eby/apa/d/you-sho...,None,(fremont / union city / newark),1br -\n 798ft2 -,None,You Should See This!
7,$1800,"[2018-10-05 1:24pm, 2018-10-24 6:59pm]",\n\n2BR / 1Ba\n923ft2\navailable oct 1\n\napar...,6715912752,2018-10-24 19:04,38.002095,https://sfbay.craigslist.org/eby/apa/d/2-bed-1...,-121.822462,(pittsburg / antioch),2br -\n 923ft2 -,QR Code Link to This Post\n\n\nNice and clean ...,2 Bed/ 1 Bath Townhome $1800/month
8,$1450,[2018-10-24 7:00pm],\n\n2BR / 2Ba\n972ft2\n\napartment\n\nw/d in u...,6732005433,2018-10-24 19:04,37.349200,https://sfbay.craigslist.org/sby/apa/d/1450-ro...,-121.953000,(santa clara),2br -\n 972ft2 -,"QR Code Link to This Post\n\n\nQuiet, professi...",$1450 / room available in 2 bed/2 bath apartment
9,$800,[],,6731999038,2018-10-24 19:03,None,https://sfbay.craigslist.org/eby/apa/d/1-bedro...,None,"(hercules, pinole, san pablo, el sob)",None,None,1 bedroom + Den hercules


In [11]:
listings_df['latitude'].isnull().sum()

774

In [13]:
listings_df['longitude'].nunique()

1436